In [11]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt


In [12]:
filename ="/pnfs/psi.ch/cms/trivcat/store/mc/Run3Summer22EENanoAODv12/VBFHHto4B_CV_1_C2V_1_C3_1_TuneCP5_13p6TeV_madgraph-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/a897cbbf-ea0b-40d5-b850-f9f24a7906a5.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=2).events()
print("Events read:", len(events))


Events read: 2
[[25, 25, 25, 25, 25, 25, 25, 25], [25, 25, 25, 25, 25, 25, 25, 25, 25, 25]]


# Use the last b quark for the matching of the b-quarks from higgs decay

In [13]:
# add column with the index
events.GenPart=ak.with_field(events.GenPart, ak.local_index(events.GenPart, axis=1), "index")
print(events.GenPart.index[0])
# get last bquark copy
isB = abs(events.GenPart.pdgId) == 5
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
bquarks_last = events.GenPart[isB & isLast & isHard]
# print(bquarks_last.pdgId, bquarks_last.genPartIdxMother)
b_quarks = bquarks_last
while True:
    print("\nloop")
    b_mother = events.GenPart[b_quarks.genPartIdxMother]
    print("old: ", "pdg", b_quarks.pdgId, "mother_idx",b_quarks.genPartIdxMother, "pt", b_quarks.pt, "index", b_quarks.index)
    print("mother: ", "pdg", b_mother.pdgId, "mother_idx",b_mother.genPartIdxMother, "pt", b_mother.pt)
    # break
    b_quarks = ak.where(abs(b_mother.pdgId) == 5, b_mother, b_quarks)
    print("new: ", "pdg", b_quarks.pdgId, "mother_idx",b_quarks.genPartIdxMother, "pt", b_quarks.pt)

    print(ak.any(abs(b_mother.pdgId) != 25, axis=1))
    if ak.all(abs(b_mother.pdgId) == 25):
        break

print(b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(b_quarks.genPartIdxMother, len(b_quarks.genPartIdxMother))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]

loop
old:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5]] mother_idx [[14, 15, 17, 13], [15, 14, 16, 17]] pt [[26.9, 28.1, 50.4, 76.2], [27.2, 60.6, 31.9, 58.6]] index [[18, 24, 26, 29], [18, 20, 25, 26]]
mother:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5]] mother_idx [[11, 11, 12, 10], [12, 12, 13, 13]] pt [[41.5, 101, 50.4, 91.5], [55.8, 66.5, 71, 60.4]]
new:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5]] mother_idx [[11, 11, 12, 10], [12, 12, 13, 13]] pt [[41.5, 101, 50.4, 91.5], [55.8, 66.5, 71, 60.4]]
[True, True]

loop
old:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5]] mother_idx [[11, 11, 12, 10], [12, 12, 13, 13]] pt [[41.5, 101, 50.4, 91.5], [55.8, 66.5, 71, 60.4]] index [[14, 15, 17, 13], [15, 14, 16, 17]]
mother:  pdg [[25, 25, 5, 25], [25, 25, 25, 25]] mother_idx [[9, 9, 10, 8], [10, 10, 11, 11]] pt [[71.8, 71.8, 132, 195], [66, 66, 43.4, 43.4]]
new:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5]] mother_idx [[11, 11, 10, 10], [12, 12

In [14]:
# order higgs pt
higgs = events.GenPart[
            (events.GenPart.pdgId == 25)
            & events.GenPart.hasFlags(["fromHardProcess"])
            & events.GenPart.hasFlags(["isLastCopy"])
        ]
higgs = higgs[ak.num(higgs.childrenIdxG, axis=2) == 2]
print(higgs.pt, higgs.childrenIdxG, higgs.children.pdgId, higgs.children.pt)
print(ak.argsort(higgs.pt,ascending=False))
print(higgs.index)
higgs=higgs[ak.argsort(higgs.pt,ascending=False)]
print(higgs.pt, higgs.childrenIdxG, higgs.children.pdgId, higgs.children.pt)
print(higgs.index)

provenance = ak.where(b_quarks.genPartIdxMother == higgs.index[:,0], 1, 2)
print(provenance)


[[195, 71.8], [66, 43.4]] [[[12, 13], [14, 15]], [[67, 68], [69, 70]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]]] [[[132, 91.5], [41.5, 101]], [[66.5, 55.8], [71, 60.4]]]
[[0, 1], [0, 1]]
[[10, 11], [12, 13]]
[[195, 71.8], [66, 43.4]] [[[12, 13], [14, 15]], [[67, 68], [69, 70]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]]] [[[132, 91.5], [41.5, 101]], [[66.5, 55.8], [71, 60.4]]]
[[10, 11], [12, 13]]
[[2, 2, 1, 1], [1, 1, 2, 2]]


# VBF partons

In [15]:
# get mother of higgs
higgs_first_copy=events.GenPart[
            (events.GenPart.pdgId == 25)
            & events.GenPart.hasFlags(["fromHardProcess"])
            & events.GenPart.hasFlags(["isFirstCopy"])]
higgs_mother = events.GenPart[higgs_first_copy.genPartIdxMother]
print(higgs_mother.pdgId)

[[2, 2], [1, 1]]


In [16]:
print(events.GenPart)
events.GenPart=ak.with_field(events.GenPart, ak.local_index(events.GenPart, axis=1), "index")

isQuark = abs(events.GenPart.pdgId) < 7
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
quarks_last = events.GenPart[isQuark & isLast & isHard]
print(quarks_last.pdgId)

[[GenParticle, GenParticle, GenParticle, ... GenParticle, GenParticle, GenParticle]]
[[5, 2, 2, -5, 5, -5], [-5, 5, 1, 5, -5, 2]]


In [25]:
vbf_quarks = quarks_last
while True:
    print("\nloop")
    vbf_quark_mother = events.GenPart[vbf_quarks.genPartIdxMother]
    # mother can't be higgs to avoid getting the quarks from higgs decay
    print("quark: ", "pdg", vbf_quarks.pdgId, "mother_idx",vbf_quarks.genPartIdxMother,"index", vbf_quarks.index)
    print("mother: ", "pdg", vbf_quark_mother.pdgId, "mother_idx",vbf_quark_mother.genPartIdxMother,  "index", vbf_quark_mother.index)
    vbf_quarks= vbf_quarks[abs(vbf_quark_mother.pdgId) != 25]
    vbf_quark_mother = vbf_quark_mother[abs(vbf_quark_mother.pdgId) != 25]
    print("quark1: ", "pdg", vbf_quarks.pdgId, "mother_idx",vbf_quarks.genPartIdxMother,"index", vbf_quarks.index)
    print("mother1: ", "pdg", vbf_quark_mother.pdgId, "mother_idx",vbf_quark_mother.genPartIdxMother,  "index", vbf_quark_mother.index)

    #check if any of the children of mother is a higgs
    mother_children = vbf_quark_mother.children
    print("children: ", "pdg", mother_children.pdgId, "mother_idx",mother_children.genPartIdxMother,  "index", mother_children.index)
    #cnotain two higgs in the children
    mother_children_isH = ak.sum((mother_children.pdgId == 25), axis=2)==2
    print("mother_children_isH", mother_children_isH)

    if ak.all(mother_children_isH):
        break
    vbf_quarks = ak.where(mother_children_isH, vbf_quarks, vbf_quark_mother)



loop
quark:  pdg [[5, 2, 2, -5, 5, -5], [-5, 5, 1, 5, -5, 2]] mother_idx [[14, 5, 4, 15, 17, 13], [15, 14, 5, 16, 17, 4]] index [[18, 20, 21, 24, 26, 29], [18, 20, 22, 25, 26, 29]]
mother:  pdg [[5, 2, 2, -5, 5, -5], [-5, 5, 1, 5, -5, 2]] mother_idx [[11, 0, 0, 11, 12, 10], [12, 12, 0, 13, 13, 0]] index [[14, 5, 4, 15, 17, 13], [15, 14, 5, 16, 17, 4]]
quark1:  pdg [[5, 2, 2, -5, 5, -5], [-5, 5, 1, 5, -5, 2]] mother_idx [[14, 5, 4, 15, 17, 13], [15, 14, 5, 16, 17, 4]] index [[18, 20, 21, 24, 26, 29], [18, 20, 22, 25, 26, 29]]
mother1:  pdg [[5, 2, 2, -5, 5, -5], [-5, 5, 1, 5, -5, 2]] mother_idx [[11, 0, 0, 11, 12, 10], [12, 12, 0, 13, 13, 0]] index [[14, 5, 4, 15, 17, 13], [15, 14, 5, 16, 17, 4]]
children:  pdg [[[5, -523, 21, 21, 521], [2], [2], [-5], ... -513], [1], [5, -513, 513], [-5], [2]]] mother_idx [[[14, 14, 14, 14, 14], [5], [4], [15], ... 14], [5], [16, 16, 16], [17], [4]]] index [[[18, 19, 22, 23, 25], [20], [21], [24], ... 21], [22], [25, 27, 28], [26], [29]]]
mother_child

In [26]:
# check if vbf quark: eta > 2.5 and delta eta >5
print(vbf_quarks.eta)
print(abs(vbf_quarks.eta[:,0] - vbf_quarks.eta[:,1]))

[[3.78, -3.41], [-3.66, 4.12]]
[7.2, 7.79]
